# 🍫Tune your RAG data pipeline and evaluate its performance

> ⚠️ This notebook can be run on your local machine or on a virtual machine and requires [Docker Compose](https://docs.docker.com/desktop/).
> Please note that it is not compatible with Google Colab as the latter does not support Docker.

In this notebook we demonstrate how to iteratively evaluate and tune a Retrieval-Augmented Generation (RAG) system using [Fondant](https://fondant.ai).

We will:

1. Set up a [Weaviate](https://weaviate.io/platform) vector store
2. Define a parameter set to test
3. Run a Fondant pipeline with those parameters to index our documents into the vector store
4. Run a Fondant pipeline with those parameters to evaluate the performance
5. Inspect the evaluation results and data between each processing step
6. Repeat step 2 - 5 until we're happy with the results

<div align="center">
<img src="../art/iteration.png" width="1000"/>
</div>

## Set up environment

> ⚠️ This section checks the prerequisites of your environment. Read any errors or warnings carefully.

Ensure a **Python between version 3.8 and 3.10** is available

In [1]:
import sys
if sys.version_info < (3, 8, 0) or sys.version_info >= (3, 11, 0):
    raise Exception(f"A Python version between 3.8 and 3.10 is required. You are running {sys.version}")

Check if **docker compose** is installed and the **docker daemon** is running

In [2]:
!docker compose version

Docker Compose version v2.19.1


Install Fondant framework

In [3]:
!pip install -q -r ../requirements.txt --disable-pip-version-check && echo "Success"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2023.12.2 which is incompatible.
Success


## Spin up the Weaviate vector store

> ⚠️ For **Apple M1/M2** chip users:
> 
> - In Docker Desktop Dashboard `Settings -> Features in development`, make sure to **un**check `Use containerd` for pulling and storing images. More info [here](https://docs.docker.com/desktop/settings/mac/#beta-features)
> - Make sure that Docker uses linux/amd64 platform and not arm64 (cell below should take care of that)

Run **Weaviate** with Docker compose

In [27]:
!docker compose -f weaviate_service/docker-compose.yaml up --detach

[+] Running 0/0
 ⠋ Network weaviate_service_default  Creating                              0.0s 
[+] Running 1/1
 ✔ Network weaviate_service_default            Created                     0.1s 
 ⠋ Container weaviate_service-contextionary-1  Creating                    0.0s 
 ⠋ Container weaviate_service-weaviate-1       Creating                    0.0s 
[+] Running 1/3
 ✔ Network weaviate_service_default            Created                     0.1s 
 ⠙ Container weaviate_service-contextionary-1  Creating                    0.1s 
 ⠙ Container weaviate_service-weaviate-1       Creating                    0.1s 
[+] Running 1/3
 ✔ Network weaviate_service_default            Created                     0.1s 
 ⠿ Container weaviate_service-contextionary-1  Starting                    0.2s 
 ⠿ Container weaviate_service-weaviate-1       Starting                    0.2s 
[+] Running 1/3
 ✔ Network weaviate_service_default            Created                     0.1s 
 ⠿ Container weaviate_service

Make sure you have **Weaviate client v3**

Make sure the vectorDB is running and accessible

In [4]:
import logging
import weaviate

try:
    local_weaviate_client = weaviate.Client("http://localhost:8081")
    logging.info("Connected to Weaviate instance")
except weaviate.WeaviateStartUpError:
    logging.error("Cannot connect to weaviate instance, is it running?")

/Users/matthias/projects/fondant/fondant-usecase-RAG/.venv/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.25.3. The latest version is 4.4.2.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


#### Indexing pipeline

Before we can evaluate data in a vector database we have to index documents. We have created a pipeline in the indexing notebook. Before you continue here, have a look at the notebook and initialise the database and the documents.  

## Evaluation Pipeline

In [9]:
import utils
base_path = "./data"
utils.create_directory_if_not_exists(base_path)
weaviate_url = f"http://{utils.get_host_ip()}:8081"
weaviate_class = "Index"

`pipeline_eval.py` evaluates retrieval performance using the questions provided in your test dataset

<div align=center>
<img src="../art/evaluation_ltr.png" width="800"/>
</div>

- [**Load eval data**](https://github.com/ml6team/fondant/tree/main/components/load_from_csv): loads the evaluation dataset (questions) from a csv file
- [**Embed questons**](https://github.com/ml6team/fondant/tree/main/components/embed_text): embeds each question as a vector, e.g. using [Cohere](https://cohere.com/embeddings)
- [**Query vector store**](https://github.com/ml6team/fondant/tree/main/components/retrieve_from_weaviate): retrieves the most relevant chunks for each question from the vector store
- [**Evaluate**](https://github.com/ml6team/fondant/tree/0.8.0/components/evaluate_ragas): evaluates the retrieved chunks for each question, e.g. using [RAGAS](https://docs.ragas.io/en/latest/index.html)
- [**Aggregate**](https://github.com/ml6team/fondant-usecase-RAG/tree/main/src/components/aggregate_eval_results): calculates aggregated results

### Create the evaluation pipeline

⚠️ If you want to use an **OpenAI** model for evaluation you will need an [API key](https://platform.openai.com/docs/quickstart) (see TODO below)

Change the arguments below if you want to run the pipeline with different parameters.

In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""

We begin by initializing our pipeline.

In [10]:
import pyarrow as pa
from fondant.pipeline import Pipeline
evaluation_pipeline = Pipeline(
        name="evaluation-pipeline",
        description="Pipeline to evaluate a RAG system",
        base_path=base_path,
)


We have created a set of evaluation questions which we will use to evaluate the retrieval performance of the RAG system. Therefore, we need to load the CSV file containing the questions. We are going to use a reusable component for this task, `load_from_csv`.

In [11]:
evaluation_set_filename = "wikitext_1000_q.csv"

load_from_csv = evaluation_pipeline.read(
    "load_from_csv",
    arguments={
        "dataset_uri": "/evaldata/" + evaluation_set_filename,
        # mounted dir from within docker as extra_volumes
        "column_separator": ";",
    },
    produces={
        "question": pa.string(),
    },
)

Afterward, we are going to embed our questions and retrieve answers from the database. Here we will once again use the reusable `embed_text` component.

In [12]:
embed_text_op = load_from_csv.apply(
    "embed_text",
    arguments={
        "model_provider": "huggingface",
        "model": "all-MiniLM-L6-v2"
    },
    consumes={
        "text": "question",
    }
)

Before we can evaluate answers, we need to retrieve these for our questions. Hence, we are building a custom lightweight component to add to our pipeline later.

In [13]:
import pandas as pd
import pyarrow as pa
from fondant.component import PandasTransformComponent
from fondant.pipeline import lightweight_component


@lightweight_component(
    produces={"retrieved_chunks": pa.list_(pa.string())},
    extra_requires=["weaviate-client==3.24.1"],
)
class RetrieveFromWeaviateComponent(PandasTransformComponent):
    def __init__(self, *, weaviate_url: str, class_name: str, top_k: int) -> None:
        import weaviate

        self.client = weaviate.Client(
            url=weaviate_url,
            additional_config=None,
            additional_headers=None,
        )
        self.class_name = class_name
        self.k = top_k

    def teardown(self) -> None:
        # Ensure the weaviate client is closed at the end of the component lifetime
        del self.client

    def retrieve_chunks_from_embeddings(self, vector_query: list):
        """Get results from weaviate database."""
        query = (
            self.client.query.get(self.class_name, ["passage"])
            .with_near_vector({"vector": vector_query})
            .with_limit(self.k)
            .with_additional(["distance"])
        )

        result = query.do()
        result_dict = result["data"]["Get"][self.class_name]
        return [retrieved_chunk["passage"] for retrieved_chunk in result_dict]

    def transform(self, dataframe: pd.DataFrame) -> pd.DataFrame:
        dataframe["retrieved_chunks"] = dataframe["embedding"].apply(self.retrieve_chunks_from_embeddings)
        return dataframe

# Add component to pipeline
retrieve_chunks = embed_text_op.apply(
    RetrieveFromWeaviateComponent,
    arguments={
        "weaviate_url": weaviate_url,
        "class_name": weaviate_class,
        "top_k": 2
    },
)

 Consumes: {'question': {'type': 'string'}, 'embedding': {'type': 'array', 'items': {'type': 'float32'}}}


`RetrieveFromWeaviateComponent` will produce `retrieved_chunks`. We aim to evaluate these chunks using RAGAS. RAGAS is an open-source library designed to assess RAG systems by leveraging LLMs. In this example, we'll use gpt-3.5-turbo. Essentially, we pass the retrieved chunks along with the answer to a LLM and ask it to judge the quality of the provided answers.

Feel free to explore the RAGAS documentation and modify the component to suit your needs. RAGAS provides support for altering the prompt and adapting it to your specific domain or language.

In [23]:
@lightweight_component(
    consumes={
        "question": pa.string(),
        "retrieved_chunks": pa.list_(pa.string()),
    },
    produces={

        "context_relevancy": pa.float32(),
    },
    extra_requires=["ragas==0.1.0"],
)
class RagasEvaluator(PandasTransformComponent):
    def __init__(self, *, open_ai_key: str) -> None:
        import os
        os.environ["OPENAI_API_KEY"] = open_ai_key

    def transform(self, dataframe: pd.DataFrame) -> pd.DataFrame:
        from datasets import Dataset
        from ragas import evaluate
        from ragas.metrics import context_relevancy
        from langchain_openai.chat_models import ChatOpenAI

        gpt_evaluator = ChatOpenAI(model_name="gpt-3.5-turbo")

        dataframe = dataframe.rename(
            columns={"retrieved_chunks": "contexts"},
        )
        
        dataset = Dataset.from_pandas(dataframe)

        result = evaluate(
            dataset,  
            metrics=[context_relevancy],
            llm=gpt_evaluator,
        )

        results_df = result.to_pandas()
        results_df = results_df.set_index(dataframe.index)

        return results_df
    
# Add component to pipeline
retriever_eval = retrieve_chunks.apply(
    RagasEvaluator,
    arguments={
        "open_ai_key": os.getenv("OPENAI_API_KEY")
    }
)

The `RAGASEvaluator` component will append two additional columns to our dataset: `context_precision` and `context_relevancy` for each question-answer pair. To evaluate the overall performance of our RAG setup, we need to aggregate these results. For demonstration purposes, we'll write the results to a file. Of course, you can export the aggregated results to any dashboard tool of your choice.

In [25]:
from fondant.component import DaskWriteComponent
import dask.dataframe as dd


@lightweight_component(
    consumes={
        "context_relevancy": pa.float32(),
    }
)
class AggregateResults(DaskWriteComponent):
    def write(self, dataframe: dd.DataFrame) -> None:
        import pandas as pd
        mean_context_relevancy = dataframe["context_relevancy"].mean()
        df = pd.DataFrame({
            "context_relevancy": mean_context_relevancy
        })

        df.to_csv("./evaldata/aggregated_results.csv")

# Add component to pipeline
retriever_eval.apply(
    AggregateResults, 
    consumes={
        "context_relevancy": "context_relevancy"
    }
)

#### Run the evaluation pipeline

In [26]:
import os
from fondant.pipeline.runner import DockerRunner
runner = DockerRunner() 
extra_volumes = [str(os.path.join(os.path.abspath('.'), "evaluation_datasets")) + ":/evaldata"]
runner.run(evaluation_pipeline, extra_volumes=extra_volumes)

INFO:root:Found reference to un-compiled pipeline... compiling
INFO:fondant.pipeline.compiler:Compiling evaluation-pipeline to .fondant/compose.yaml
INFO:fondant.pipeline.compiler:Base path found on local system, setting up ./data as mount volume
INFO:fondant.pipeline.pipeline:Sorting pipeline component graph topologically.
INFO:fondant.pipeline.pipeline:All pipeline component specifications match.
INFO:fondant.pipeline.compiler:Compiling service for load_from_csv
INFO:fondant.pipeline.compiler:Compiling service for embed_text
INFO:fondant.pipeline.compiler:Compiling service for retrievefromweaviatecomponent
INFO:fondant.pipeline.compiler:Compiling service for ragasevaluator
INFO:fondant.pipeline.compiler:Compiling service for aggregateresults
INFO:fondant.pipeline.compiler:Successfully compiled to .fondant/compose.yaml
 load_from_csv Pulling 
 ragasevaluator Pulling 
 aggregateresults Pulling 
 embed_text Pulling 
 retrievefromweaviatecomponent Pulling 


Starting pipeline run...


 retrievefromweaviatecomponent Pulled 
 ragasevaluator Pulled 
 load_from_csv Pulled 
 embed_text Pulled 
 aggregateresults Pulled 
 Container evaluation-pipeline-load_from_csv-1  Recreate
 Container evaluation-pipeline-load_from_csv-1  Recreated
 Container evaluation-pipeline-embed_text-1  Recreate
 Container evaluation-pipeline-embed_text-1  Recreated
 Container evaluation-pipeline-retrievefromweaviatecomponent-1  Recreate
 Container evaluation-pipeline-retrievefromweaviatecomponent-1  Recreated
 Container evaluation-pipeline-ragasevaluator-1  Recreate
 Container evaluation-pipeline-ragasevaluator-1  Recreated
 Container evaluation-pipeline-aggregateresults-1  Recreate
 Container evaluation-pipeline-aggregateresults-1  Recreated


Attaching to evaluation-pipeline-aggregateresults-1, evaluation-pipeline-embed_text-1, evaluation-pipeline-load_from_csv-1, evaluation-pipeline-ragasevaluator-1, evaluation-pipeline-retrievefromweaviatecomponent-1


evaluation-pipeline-load_from_csv-1                  | [2024-02-08 13:30:29,549 | fondant.cli | INFO] Component `CSVReader` found in module main
evaluation-pipeline-load_from_csv-1                  | [2024-02-08 13:30:29,554 | fondant.component.executor | INFO] Dask default local mode will be used for further executions.Our current supported options are limited to 'local' and 'default'.
evaluation-pipeline-load_from_csv-1                  | [2024-02-08 13:30:29,558 | fondant.component.executor | INFO] Skipping component execution
evaluation-pipeline-load_from_csv-1                  | [2024-02-08 13:30:29,561 | fondant.component.executor | INFO] Matching execution detected for component. The last execution of the component originated from `evaluation-pipeline-20240206105318`.
evaluation-pipeline-load_from_csv-1                  | [2024-02-08 13:30:29,566 | fondant.component.executor | INFO] Saving output manifest to /data/evaluation-pipeline/evaluation-pipeline-20240208143024/load_from_

...

[                                        ] | 0% Completed | 1.80 s
[                                        ] | 0% Completed | 1.91 s
[                                        ] | 0% Completed | 2.01 s
[                                        ] | 0% Completed | 2.11 s
[                                        ] | 0% Completed | 2.21 s
[                                        ] | 0% Completed | 2.32 s
[                                        ] | 0% Completed | 2.42 s
[                                        ] | 0% Completed | 2.53 s
[                                        ] | 0% Completed | 2.64 s
[                                        ] | 0% Completed | 2.74 s


evaluation-pipeline-ragasevaluator-1                 | [2024-02-08 13:31:01,381 | httpx | INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
evaluation-pipeline-ragasevaluator-1                 | [2024-02-08 13:31:01,394 | httpx | INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[                                        ] | 0% Completed | 2.84 s
[                                        ] | 0% Completed | 2.94 s
[                                        ] | 0% Completed | 3.05 s
[                                        ] | 0% Completed | 3.15 s
[                                        ] | 0% Completed | 3.26 s
[                                        ] | 0% Completed | 3.36 s
[                                        ] | 0% Completed | 3.46 s
[                                        ] | 0% Completed | 3.56 s


evaluation-pipeline-ragasevaluator-1                 | [2024-02-08 13:31:02,236 | httpx | INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[                                        ] | 0% Completed | 3.67 s
[                                        ] | 0% Completed | 3.77 s
[                                        ] | 0% Completed | 3.87 s
[                                        ] | 0% Completed | 3.97 s
[                                        ] | 0% Completed | 4.08 s
[                                        ] | 0% Completed | 4.18 s
[                                        ] | 0% Completed | 4.28 s
[                                        ] | 0% Completed | 4.38 s
[                                        ] | 0% Completed | 4.49 s
[                                        ] | 0% Completed | 4.59 s
[                                        ] | 0% Completed | 4.70 s
[                                        ] | 0% Completed | 4.80 s
[                                        ] | 0% Completed | 4.90 s
[                                        ] | 0% Completed | 5.01 s
[                                        ] | 0% Completed | 5.

#### Show evaluation results

In [22]:
import pandas as pd
df = pd.read_csv("./evaluation_dataset/aggregated_results.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: './evaluation_dataset/aggregated_results.csv'

## Explore data

You can also check your data and results at each step in the pipelines using the **Fondant data explorer**. The first time you run the data explorer, you need to download the docker image which may take a minute. Then you can access the data explorer at: **http://localhost:8501/**

Enjoy the exploration! 🍫 

Press the ◼️ in the notebook toolbar to **stop the explorer**.

In [ ]:
from fondant.explore import run_explorer_app
run_explorer_app(base_path=base_path)

To stop the Explore, run the cell below.

In [ ]:
from fondant.explore import stop_explorer_app
stop_explorer_app()

## Clean up your environment

After your pipeline run successfully, you can **clean up** your environment and stop the weaviate database.

In [ ]:
!docker compose -f weaviate/docker-compose.yaml down

## Feedback

Please share your experience or **let us know how we can improve** through our 
* [**Discord**](https://discord.gg/HnTdWhydGp) 
* [**GitHub**](https://github.com/ml6team/fondant)

And of course feel free to give us a [**star** ⭐](https://github.com/ml6team/fondant) if you like what we are doing!